In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit

import numpy as np
import qutip as qt
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import PI_GRAPE, GRAPE
import matplotlib.pyplot as plt
import h5py

In [ ]:
q_dim = 3
c_dim = 12
DAC_time_resolution = 2 # in ns

# In GHz = cycles / ns
anharm = -.180
kerr = -1e-5
chi = -0.4e-3
chi_prime = 1e-5
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
gf = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 0) * qt.basis(q_dim, 2).dag())
fg = gf.dag()
ad = a.dag()
bd = b.dag()
H0 = (anharm/2) * bd * bd * b * b
H0 = (chi) * ad * a * bd * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi_prime / 6) * ad * ad * a * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(b + bd), 1j*D*(b - bd)]

init_states = []
final_states = []

cutoff = 6
for k in np.arange(cutoff + 1):
    init_states.append(qt.tensor(qt.basis(c_dim, k), qt.basis(q_dim, 0)))

    if k % 2 == 0:
        final_states.append(qt.tensor(qt.basis(c_dim, k), qt.basis(q_dim, 1)))
    else:
        final_states.append(qt.tensor(qt.basis(c_dim, k), qt.basis(q_dim, 0)))


In [ ]:
synth_params = {
    'N_blocks': 751, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 15, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 3000, #number of epochs before termination
    'epoch_size' : 5, #number of adam steps per epoch
    'learning_rate' : 0.1, #adam learning rate
    'term_fid' : 0.999, #0.995, #terminal fidelitiy
    'dfid_stop' : 1e-5/2, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'Parity GRAPE', #name for printing and saving
    'use_phase' : True,
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}

# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'inplace' : False, # true uses less memory, but is slower. Just use false
    'scale' : 1.0, # range of DAC amplitudes for initial random waves
    'bandwidth' : 0.05,
    'ringup' : 40,
    'gatesynthargs': synth_params
}



GRAPE_gate_set = GRAPE(**gate_set_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(GRAPE_gate_set)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
GRAPE_gate_set.best_fidelity()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
# Note that you will not want to use the performance profiler while using 'inplace' mode. You will run out of memory
opt.optimize()#logdir='logs')

In [ ]:
GRAPE_gate_set.print_info()

In [ ]:
f = h5py.File('Parity GRAPE.h5', 'r')
pulse_obj = f[list(f.keys())[len(list(f.keys())) - 1]] # get the latest key
fids = pulse_obj['fidelities']
# list(f.keys())
pulse_idx = np.argmax(np.amax(fids, axis=0))
controls = []
for k in range(1):
    controls.append(np.array(pulse_obj['I' + str(k)][-1, pulse_idx, :]))
    controls.append(np.array(pulse_obj['Q' + str(k)][-1, pulse_idx, :]))
DAC_time_resolution = 2 # in ns
pulse_len = controls[0].shape[0] * DAC_time_resolution
times = np.arange(pulse_len, step = DAC_time_resolution, dtype=float)
fids[-1, pulse_idx]

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(times, controls[0], 'b-')
plt.plot(times, controls[1], 'b:')
plt.xlabel(r'time ns')
plt.ylabel(r'Drive / $\chi$')

In [ ]:
plt.figure(figsize=(9,3))
plt.plot(np.fft.fftfreq(len(controls[0]), DAC_time_resolution) * 1000, np.abs(np.fft.fft(controls[0])), 'b-')
plt.plot(np.fft.fftfreq(len(controls[0]), DAC_time_resolution) * 1000, np.abs(np.fft.fft(controls[1])), 'b:')
plt.xlabel(r'MHz')
plt.xlim(-100, 100)
plt.ylabel(r'Mag')
# plt.yscale('log')

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs